### Collecting Libraries

In [615]:
import re
import pandas as pd
import numpy as np

# Pakai PySastrawi buat bantu stemming
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

# Pakai nltk buat ambil stopwords nya
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

import matplotlib.pyplot as plt

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\dell\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


#### Load to-be-cleansed file and supporting file

In [616]:
# Main file, di-define sebagai df_data
colnames = ['text', 'sentiment']
df_data = pd.read_csv('train_preprocess.tsv', sep='\t', header=None, names=colnames)
print('Done')

Done


In [617]:
# Kamus alay
kamus_label = ['alay', 'tidak alay']
kamus_alay = pd.read_csv('new_kamusalay.csv', header=None, encoding='latin-1', names=kamus_label)
print('Done')

Done


In [618]:
df_data.head()

,text,sentiment
0,warung ini dimiliki oleh pengusaha pabrik tahu...,positive
1,mohon ulama lurus dan k212 mmbri hujjah partai...,neutral
2,lokasi strategis di jalan sumatera bandung . t...,positive
3,betapa bahagia nya diri ini saat unboxing pake...,positive
4,duh . jadi mahasiswa jangan sombong dong . kas...,negative


In [619]:
df_data.isnull().sum() # Cek ada null atau nggak

text         0
sentiment    0
dtype: int64

In [620]:
df_data.describe() # Iseng aja liat data seperti apa kalau didescribe

,text,sentiment
count,11000,11000
unique,10933,3
top,kesal,positive
freq,4,6416


In [621]:
df_data.info() # Mastiin lagi aja gak ada null

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11000 entries, 0 to 10999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   text       11000 non-null  object
 1   sentiment  11000 non-null  object
dtypes: object(2)
memory usage: 172.0+ KB


#### Stopwords

In [622]:
list_stopword = stopwords.words('indonesian')
list_stopword

['ada',
 'adalah',
 'adanya',
 'adapun',
 'agak',
 'agaknya',
 'agar',
 'akan',
 'akankah',
 'akhir',
 'akhiri',
 'akhirnya',
 'aku',
 'akulah',
 'amat',
 'amatlah',
 'anda',
 'andalah',
 'antar',
 'antara',
 'antaranya',
 'apa',
 'apaan',
 'apabila',
 'apakah',
 'apalagi',
 'apatah',
 'artinya',
 'asal',
 'asalkan',
 'atas',
 'atau',
 'ataukah',
 'ataupun',
 'awal',
 'awalnya',
 'bagai',
 'bagaikan',
 'bagaimana',
 'bagaimanakah',
 'bagaimanapun',
 'bagi',
 'bagian',
 'bahkan',
 'bahwa',
 'bahwasanya',
 'baik',
 'bakal',
 'bakalan',
 'balik',
 'banyak',
 'bapak',
 'baru',
 'bawah',
 'beberapa',
 'begini',
 'beginian',
 'beginikah',
 'beginilah',
 'begitu',
 'begitukah',
 'begitulah',
 'begitupun',
 'bekerja',
 'belakang',
 'belakangan',
 'belum',
 'belumlah',
 'benar',
 'benarkah',
 'benarlah',
 'berada',
 'berakhir',
 'berakhirlah',
 'berakhirnya',
 'berapa',
 'berapakah',
 'berapalah',
 'berapapun',
 'berarti',
 'berawal',
 'berbagai',
 'berdatangan',
 'beri',
 'berikan',
 'berikut'

In [623]:
print(len(list_stopword))

758


In [624]:
list_stopword.extend(["yg", "dg", "rt", "dgn", "ny", "d", 'klo', 
                       'kalo', 'amp', 'biar', 'bikin', 'bilang', 
                       'gak', 'ga', 'krn', 'nya', 'nih', 'sih', 
                       'si', 'tau', 'tdk', 'tuh', 'utk', 'ya', 
                       'jd', 'jgn', 'sdh', 'aja', 'n', 't', 
                       'nyg', 'hehe', 'pen', 'u', 'nan', 'loh', 'rt',
                       '&amp', 'yah', 'nya', 'ber', 'banget', 'kali'])

In [625]:
list_stopword
print(len(list_stopword))

801


In [626]:
# Di internet sih ini buat hilangin duplicated data di dalam list
list_stopword = list(dict.fromkeys(list_stopword))
print(len(list_stopword))

798


In [627]:
# List nya kita ubah jadi set
list_stopword = set(list_stopword)
list_stopword

{'&amp',
 'ada',
 'adalah',
 'adanya',
 'adapun',
 'agak',
 'agaknya',
 'agar',
 'aja',
 'akan',
 'akankah',
 'akhir',
 'akhiri',
 'akhirnya',
 'aku',
 'akulah',
 'amat',
 'amatlah',
 'amp',
 'anda',
 'andalah',
 'antar',
 'antara',
 'antaranya',
 'apa',
 'apaan',
 'apabila',
 'apakah',
 'apalagi',
 'apatah',
 'artinya',
 'asal',
 'asalkan',
 'atas',
 'atau',
 'ataukah',
 'ataupun',
 'awal',
 'awalnya',
 'bagai',
 'bagaikan',
 'bagaimana',
 'bagaimanakah',
 'bagaimanapun',
 'bagi',
 'bagian',
 'bahkan',
 'bahwa',
 'bahwasanya',
 'baik',
 'bakal',
 'bakalan',
 'balik',
 'banget',
 'banyak',
 'bapak',
 'baru',
 'bawah',
 'beberapa',
 'begini',
 'beginian',
 'beginikah',
 'beginilah',
 'begitu',
 'begitukah',
 'begitulah',
 'begitupun',
 'bekerja',
 'belakang',
 'belakangan',
 'belum',
 'belumlah',
 'benar',
 'benarkah',
 'benarlah',
 'ber',
 'berada',
 'berakhir',
 'berakhirlah',
 'berakhirnya',
 'berapa',
 'berapakah',
 'berapalah',
 'berapapun',
 'berarti',
 'berawal',
 'berbagai',
 'b

In [628]:
bukan_stopword = {'baik', 'masalah', 'yakin', 'tidak', 'pantas', 'lebih'}

In [629]:
final_stopword = set([word for word in list_stopword if word not in bukan_stopword])

In [630]:
len(final_stopword)

792

In [631]:
# Iseng aja liat isi kamus alay
kamus_alay.head()

,alay,tidak alay
0,anakjakartaasikasik,anak jakarta asyik asyik
1,pakcikdahtua,pak cik sudah tua
2,pakcikmudalagi,pak cik muda lagi
3,t3tapjokowi,tetap jokowi
4,3x,tiga kali


In [632]:
tes = re.findall('t3tapjokowi', str(kamus_alay['alay']))
print(tes)

['t3tapjokowi']


#### Data Cleansing :
- Normalizing words
- Stemming
- Removing Stopwords
- Final cleaning using RegEx

Membuat fungsi untuk melakukan cleaning

In [633]:
# Normalisasi kata alay (kalau ada), pake kamus alay
kamus_alay_map = dict(zip(kamus_alay['alay'], kamus_alay['tidak alay']))
def normalisasi_alay(text):
    return ' '.join([kamus_alay_map[word] if word in kamus_alay_map else word for word in text.split(' ')])
    # text = ' '.join(map(str, [kamus_alay[word] if word in kamus_alay else word for word in text.split(' ')]))
    # return text

In [634]:
# Stemming pake PySastrawi
factory = StemmerFactory()
stemmer = factory.create_stemmer()

def stemming(text):
    text = stemmer.stem(text)
    return text
# def stemming(text):
#     return stemmer.stem(text)

In [635]:
# Hilangin stopword
def remove_stopword(text):
    text = ' '.join(['' if word in final_stopword else word for word in text.split(' ')])
    text = text.strip()
    return text

In [636]:
# Hilangin url pattern, whitespace berlebih, newline, tab, dan simbol-simbol
def cleaning(text):
    url_pattern = r'((http|https)\:\/\/)?[a-zA-Z0-9\.\/\?\:@\-_=#]+\.([a-zA-Z]){2,6}([a-zA-Z0-9\.\&\/\?\:@\-_=#])*'
    text = re.sub(url_pattern, " ", text)
    text = re.sub(r'[^a-zA-Z0-9]', " ", text)
    text = re.sub(r'[^\d+\s|\s\d+\s|\s\d+$]', " ", text)
    text = re.sub(r'[^a-zA-Z?]', " ", text)
    text = re.sub(r' {2,}', " ", text)
    text = re.sub('\n',' ',text)
    text = re.sub('\t',' ',text)
    text = re.sub('user',' ',text)
    text = text.lower()
    return text

In [637]:
# Pakai fungsi normalisasi alay
for data in [df_data]:
    data['text_normalized'] = data['text'].apply(normalisasi_alay)
print('Done')

Done


In [638]:
idx = 787
print(df_data['text'][idx])
print(df_data['text_normalized'][idx])

kebaikan
kebaikan


In [639]:
# Pakai fungsi stemming
for data in [df_data]:
    data['text_stem'] = data['text_normalized'].apply(stemming)
print('Done')

Done


In [640]:
print(df_data['text_normalized'][idx])
print(df_data['text_stem'][idx])

kebaikan
baik


In [641]:
# Pakai fungsi hilangin stopword
for data in [df_data]:
    data['text_wsw'] = data['text_stem'].apply(remove_stopword)
print('Done')

Done


In [642]:
print(df_data['text_stem'][idx])
print(df_data['text_wsw'][idx])

baik
baik


In [643]:
# Pakai fungsi cleaning
for data in [df_data]:
    data['text_clean'] = data['text_wsw'].apply(cleaning)
print('Done')

TypeError: unsupported operand type(s) for &: 'str' and 'RegexFlag'

In [ ]:
print(df_data['text_wsw'][idx])
print(df_data['text_clean'][idx])

baik
['baik']


In [ ]:
# Mengurutkan kolom biar enak dilihat
df_data = df_data[['text', 'text_normalized', 'text_stem', 'text_wsw', 'text_clean', 'sentiment']]
df_data.head()

,text,text_normalized,text_stem,text_wsw,text_clean,sentiment
0,warung ini dimiliki oleh pengusaha pabrik tahu...,warung ini dimiliki oleh pengusaha pabrik tahu...,warung ini milik oleh usaha pabrik tahu yang s...,warung milik usaha pabrik puluh kenal ...,['warung milik usaha pabrik puluh kenal ...,positive
1,mohon ulama lurus dan k212 mmbri hujjah partai...,mohon ulama lurus dan k212 mmbri hujjah partai...,mohon ulama lurus dan k212 mmbri hujjah partai...,mohon ulama lurus k212 mmbri hujjah partai ...,"['mohon ulama lurus ', 'k212', ' mmbri hujjah...",neutral
2,lokasi strategis di jalan sumatera bandung . t...,lokasi strategis di jalan sumatra bandung . te...,lokasi strategis di jalan sumatra bandung temp...,lokasi strategis jalan sumatra bandung nyam...,['lokasi strategis jalan sumatra bandung ny...,positive
3,betapa bahagia nya diri ini saat unboxing pake...,betapa bahagia nya diri ini saat unboxing pake...,betapa bahagia nya diri ini saat unboxing pake...,betapa bahagia unboxing paket barang bag...,['betapa bahagia unboxing paket barang b...,positive
4,duh . jadi mahasiswa jangan sombong dong . kas...,aduh . jadi mahasiswa jangan sombong dong . ka...,aduh jadi mahasiswa jangan sombong dong kasih ...,aduh mahasiswa sombong kasih kartu kuning ...,['aduh mahasiswa sombong kasih kartu kuning...,negative


In [ ]:
# Compare text awal banget vs yang sudah lewatin semua tahap cleansing
print(df_data['text'][idx])
print(df_data['text_clean'][idx])

kebaikan
['baik']


### Save new csv file for clean text

In [ ]:
df_data = df_data.drop(['text', 'text_normalized', 'text_stem', 'text_wsw'], axis=1)

In [ ]:
df_data.head()

,text_clean,sentiment
0,['warung milik usaha pabrik puluh kenal ...,positive
1,"['mohon ulama lurus ', 'k212', ' mmbri hujjah...",neutral
2,['lokasi strategis jalan sumatra bandung ny...,positive
3,['betapa bahagia unboxing paket barang b...,positive
4,['aduh mahasiswa sombong kasih kartu kuning...,negative


In [ ]:
print(df_data['text_clean'][787])

['baik']


In [ ]:
# df_data.to_csv('text_clean.csv', index=False)

In [ ]:
txt = "The rain in Spain"
x = re.search("^The.*Spain$", txt)

In [ ]:
x

<re.Match object; span=(0, 17), match='The rain in Spain'>